<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import os
import sys
from pathlib import Path
import multiprocessing as mp
from skimage.data import imread
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from itertools import chain
from multiprocessing import cpu_count
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


class IEEEPreprocessor:
    def __init__(self, train_dir, test_dir, img_resize=(32, 32), validation_split=0.2, process_count=cpu_count()):
        """
        This class is used by the classifier to preprocess data, don't forget to call the init() method
        after an object from this class gets created.
        :param train_dir: string
            The directory of training files
            
        :param test_dir: string
            The directory of test files
            
        :param img_resize: tuple(int, int)
            The resize size of the original image given by the file_path argument
            
        :param process_count: int
            The number of process you want to use to process the data.
            If you run into issues, lower this number. Its default value is equal to the number
            of cores of your CPU
        """
        
        self.process_count = process_count
        self.img_resize = img_resize
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.

In [2]:
train_dir = Path('/home/abanihi/Documents/deep-data/kaggle/IEEE-camera-model/train')

In [3]:
test_dir = Path('/home/abanihi/Documents/deep-data/kaggle/IEEE-camera-model/test')

In [4]:
cameras = os.listdir(train_dir)
cameras

['HTC-1-M7',
 'iPhone-6',
 'iPhone-4s',
 'Samsung-Galaxy-Note3',
 'Motorola-Nexus-6',
 'Motorola-Droid-Maxx',
 'LG-Nexus-5x',
 'Sony-NEX-7',
 'Samsung-Galaxy-S4',
 'Motorola-X']

In [5]:
train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_dir / camera)):
        train_images.append((camera, fname))

In [6]:
train_data = pd.DataFrame(train_images, columns=['camera', 'fname'])

In [7]:
train_data.head()

,camera,fname
0,HTC-1-M7,(HTC-1-M7)1.jpg
1,HTC-1-M7,(HTC-1-M7)10.jpg
2,HTC-1-M7,(HTC-1-M7)100.jpg
3,HTC-1-M7,(HTC-1-M7)101.jpg
4,HTC-1-M7,(HTC-1-M7)102.jpg


In [8]:
train_data.shape

(2750, 2)

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2750 entries, 0 to 2749
Data columns (total 2 columns):
camera    2750 non-null object
fname     2750 non-null object
dtypes: object(2)
memory usage: 43.0+ KB


In [10]:
test_images = []
for fname in sorted(os.listdir(test_dir)):
    test_images.append(fname)

In [11]:
test_data = pd.DataFrame(test_images, columns=['fname'])

In [12]:
test_data.head()

,fname
0,img_0002a04_manip.tif
1,img_001e31c_unalt.tif
2,img_00275cf_manip.tif
3,img_0034113_unalt.tif
4,img_00344b7_unalt.tif


In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 1 columns):
fname    2640 non-null object
dtypes: object(1)
memory usage: 20.7+ KB


In [20]:
def color_stats(queue, iolock):
    while True:
        img_path = queue.get()
        if img_path is None:
            break
            
        if type(img_path) is tuple:
            img = imread(train_dir / img_path[0] / img_path[1])
            key = img_path[1]
            
        else:
            img = imread(test_dir / img_path)
            key = img_path
            
        # Some images read return info in a 2nd dim. We only want the first dim
        if img.shape == (2, ):
            img = img[0]
            
        color_info[key] = (img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean(),
                           img[:, :, 0].std(),  img[:, :, 1].std(),  img[:, :, 2].std())

            
            

In [14]:
img = imread('/home/abanihi/Documents/deep-data/kaggle/IEEE-camera-model/train/HTC-1-M7/(HTC-1-M7)1.jpg')

In [15]:
img.shape

(2688, 1520, 3)

In [22]:
cols = ['a0', 'a1', 'a2', 's0', 's1', 's2']

for col in cols:
    train_data[col] = None
    test_data[col] = None

In [23]:
train_data.head()

,camera,fname,a0,a1,a2,s0,s1,s2
0,HTC-1-M7,(HTC-1-M7)1.jpg,None,None,None,None,None,None
1,HTC-1-M7,(HTC-1-M7)10.jpg,None,None,None,None,None,None
2,HTC-1-M7,(HTC-1-M7)100.jpg,None,None,None,None,None,None
3,HTC-1-M7,(HTC-1-M7)101.jpg,None,None,None,None,None,None
4,HTC-1-M7,(HTC-1-M7)102.jpg,None,None,None,None,None,None


In [24]:
NCORE = cpu_count()
NCORE

4

In [25]:
color_info = mp.Manager().dict()
color_info

<DictProxy object, typeid 'dict' at 0x7f101f9c81d0>

In [26]:
# Use a queue since the image reading is a bottlenect
queue = mp.Queue(maxsize=NCORE)
iolock = mp.Lock()

In [27]:
pool = mp.Pool(NCORE, initializer=color_stats, initargs=(queue, iolock))

In [28]:
for image in train_images:
    queue.put(image) # Blocks until queue below its max size

In [29]:
for image in test_images:
    queue.put(image)

In [30]:
# Tell workers we are done
for _ in range(NCORE):
    queue.put(None)

In [31]:
pool.close()
pool.join()

In [32]:
color_info = dict(color_info)

In [33]:
for n, col in enumerate(cols):
    train_data[col] = train_data['fname'].apply(lambda x: color_info[x][n])
    test_data[col] = test_data['fname'].apply(lambda x: color_info[x][n])

In [34]:
train_data.head()

,camera,fname,a0,a1,a2,s0,s1,s2
0,HTC-1-M7,(HTC-1-M7)1.jpg,118.223227,115.003896,111.967798,57.210818,54.533104,64.853878
1,HTC-1-M7,(HTC-1-M7)10.jpg,128.141141,122.825341,107.254211,56.619943,55.002498,59.375556
2,HTC-1-M7,(HTC-1-M7)100.jpg,98.610660,89.714463,82.452863,68.868366,73.298074,73.463918
3,HTC-1-M7,(HTC-1-M7)101.jpg,109.477162,104.011235,102.435257,46.961676,50.286467,54.133753
4,HTC-1-M7,(HTC-1-M7)102.jpg,105.730427,70.830199,103.631270,17.310780,18.811962,15.008880


In [35]:
test_data.head()

,fname,a0,a1,a2,s0,s1,s2
0,img_0002a04_manip.tif,187.380531,181.926029,173.161304,34.441726,34.237523,28.067430
1,img_001e31c_unalt.tif,171.232990,140.817070,121.201546,66.119656,69.017232,71.617331
2,img_00275cf_manip.tif,86.310349,85.689690,79.065269,52.221585,54.244189,53.314759
3,img_0034113_unalt.tif,152.006332,140.768970,132.285995,70.564771,65.590737,58.753384
4,img_00344b7_unalt.tif,95.538319,100.303024,92.712677,70.115464,65.860571,64.039072


In [36]:
train_data.to_csv('train.csv')

In [37]:
test_data.to_csv('test.csv')

In [ ]:
train_data.to_n